# SQL Generation Agent

In [ ]:
agent_instruction = """
You are an AI assistant specialized in converting natural language queries into SQL statements. 
Your primary function is to accurately translate user-provided questions into executable SQL queries 
by dynamically retrieving the necessary database schema and validating the queries using provided tools.

<tools>
get_schema(table_name)
Description: Connects to the database and retrieves the schema of the specified tables, including column names, data types, primary keys, foreign keys, and relationships.
Usage: Use this function to obtain detailed information about tables involved in the user's query.

sql_validation(sql_query)
Description: Executes the provided SQL query against the database and returns the result set or any error messages.
Usage: Use this function to validate the correctness and effectiveness of the generated SQL queries.
</tools>

You follow the steps as below to complete the task.
-Schema Identification:
Dynamically determine the database schema by identifying relevant tables and their relationships based on the user's query.

-SQL Generation:
Translate the natural language query into a syntactically correct and efficient SQL statement.

-SQL Validation:
Execute the generated SQL query to ensure it retrieves the intended results.
Analyze execution outcomes to identify discrepancies or errors.

-Error Correction:
Modify and optimize the SQL query if validation reveals issues, ensuring alignment with the user's intent.
"""

## Prerequisites
Requires knowledgebase ID and sqlite db created from data-prep-course-recommendation-agent.ipynb
Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [ ]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

Let's now check the boto3 version to ensure the correct version has been installed. Your version should be greater than or equal to 1.34.90.

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

In [ ]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Let's now create the boto3 clients for the required AWS services

In [ ]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Next we can set some configuration variables for the agent and for the lambda function being created

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

## Agent creation

In [ ]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "text2sql-agent-testing"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for sql generation"
agent_instruction = agent_instruction
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'


text2sql_action_group_name = "Text2SqlActionGroup"
text2sql_action_group_description = '''
Generate and validate SQL
'''

In [ ]:
text2sql_lambda_function_name = f'{agent_name}-text2sql-{suffix}'

### Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it. 

In [ ]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

Once the needed IAM role is created, we can use the Bedrock Agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. Later, we will prepare and use the agent.

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)

Let's now store the agent id in a local variable to use it on subsequent steps.

In [ ]:
text2sql_agent_id = response['agent']['agentId']
bedrock_agent_client.prepare_agent(agentId=text2sql_agent_id)
time.sleep(1)
text2sql_agent_alias = bedrock_agent_client.create_agent_alias(
            agentAliasName="text2sql", agentId=text2sql_agent_id
        )
text2sql_agent_alias_id = text2sql_agent_alias["agentAlias"]["agentAliasId"]
text2sql_agent_alias_arn = text2sql_agent_alias["agentAlias"][
            "agentAliasArn"
        ]

In [ ]:
%store text2sql_agent_id
%store text2sql_agent_alias_id
%store text2sql_agent_alias_arn

### Creating Lambda function

In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

In [ ]:
iam_client.get_role(RoleName=lambda_function_role)

In [ ]:
# Package up the lambda function code (course schedule lambda)
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("text2sql_lambda_function.py")
z.write("../porterville_academic.db")
z.close()
zip_content = s.getvalue()

lambda_function_2 = lambda_client.create_function(
    FunctionName=text2sql_lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='text2sql_lambda_function.lambda_handler'
)


In [ ]:
lambda_iam_role['Role']['Arn']

### Create Agent Action Groups

In [ ]:
text2sql_functions = [
    {
        'name': 'get_schema',
        'description': 'get table schema',
    },
    {
        'name': 'sql_validation',
        'description': 'execute sql query to validate its correctness',
        'parameters': {
            "query": {
                "description": "sql validation",
                "required": True,
                "type": "string"
            }
        }
    }
]

In [ ]:
# Pause to make sure agent is created
# time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=text2sql_agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_2['FunctionArn']
    },
    actionGroupName=text2sql_action_group_name,
    functionSchema={
        'functions': text2sql_functions
    },
    description=text2sql_action_group_description
)


In [ ]:
agent_action_group_response

In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=text2sql_lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{text2sql_agent_id}",
)

In [ ]:
# test agent from console